In [2]:
import os

import numpy as np
np.set_printoptions(suppress=True)
import pandas as pd

from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

print("Package is ready.")

Package is ready.


# 2020 Summer

In [3]:
pg_df = []

In [4]:
temp_df = pd.read_csv('./results/2020_S/node_length.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_len'])

pg_df.append(temp_df)

In [5]:
temp_df = pd.read_csv('./results/2020_S/node_diameter.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['node_dia'])

pg_df.append(temp_df)

In [6]:
temp_df = pd.read_csv('./results/2020_S/plant_height.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['plant_h'])

pg_df.append(temp_df)

In [7]:
temp_df = pd.read_csv('./results/2020_S/leaf_area.csv', index_col='Unnamed: 0')
temp_df.columns = temp_df.columns.astype('int64')

last_day = pd.concat([temp_df.loc['2020-07-07'].dropna(),
                      temp_df.loc['2020-07-08'].dropna(),
                      temp_df.loc['2020-07-09'].dropna()]).reset_index().drop('index', axis=1)
last_day.index = list(last_day.index)

temp_index = temp_df.index
temp_df = pd.concat([temp_df.loc[:'2020-06-17'], last_day.T])/10000
temp_df.index = list(temp_index[:-3]) + ['2020-07-03']
temp_df.index = pd.DatetimeIndex(temp_df.index)

l = []
l_i = []
for _ in range(temp_df.shape[0]):
    ll = temp_df.iloc[_].dropna()
    l.extend(ll.values)
    l_i.extend([temp_df.index[_]]*ll.shape[0])
temp_df = pd.DataFrame(l, index=l_i, columns=['leaf_area'])

pg_df.append(temp_df)

In [8]:
pg_df = pd.concat(pg_df, axis=1)
pg_df['DAT'] = [27]*3 + [55]*3 + [86]*3 + [112]*4 + [128]*20
pg_df['date'] = pg_df.index
pg_df = pg_df.set_index(['DAT'])

In [9]:
pg_reg_df = []
plot_x = [_ for _ in range(14, 129)]
date_x = pd.date_range('2020-03-11', '2020-07-03', freq='1d')

In [10]:
def sigmoid(x, L, k, x0, b):
    return L / (b + np.exp(-k*(x-x0)))

In [11]:
pg_df.iloc[-18, 0] = np.nan
pg_df.iloc[-2, 1] = np.nan
pg_df.iloc[-6, 1] = np.nan
pg_df.iloc[-21, 0] -= 3
pg_df = pg_df.dropna()

In [12]:
x = pg_df.index
y = pg_df['node_len']
polreg = np.poly1d(np.polyfit(x,y,3))
print(r2_score(y, polreg(x)))

plot_y = polreg(plot_x)
pg_reg_df.append(plot_y)

0.7185346190241422


In [13]:
polreg.c

array([ 0.00002078, -0.00446117,  0.28968234, -1.87530395])

In [14]:
x = pg_df.index
y = pg_df['node_dia']
polreg = np.poly1d(np.polyfit(x,y,3))
print(r2_score(y, polreg(x)))

plot_y = polreg(plot_x)
pg_reg_df.append(plot_y)

0.8822503506478436


In [15]:
polreg.c

array([ 0.00001451, -0.00276196,  0.16731076,  1.47825226])

In [16]:
x = pg_df.index
y = pg_df['plant_h']
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y)

0.9037982257172344


In [17]:
popt

array([ 6.60628375,  0.07970437, -3.30204321,  0.03302972])

In [18]:
x = pg_df.index
y = pg_df['leaf_area']*1000
popt, pcov = curve_fit(sigmoid, x, y, method='trf')
print(r2_score(y, sigmoid(x, *popt)))

plot_y = sigmoid(plot_x, *popt)
pg_reg_df.append(plot_y/1000)

0.8882709607841659


In [25]:
popt/1000

array([ 0.01394857,  0.00007984, -0.00623317,  0.00001069])

In [22]:
popt

array([13.94856502,  0.07983658, -6.23317329,  0.01069047])

In [20]:
pg_reg_df = pd.DataFrame(pg_reg_df).T
pg_reg_df.columns = ['node_len', 'node_dia', 'plant_h', 'leaf_area']
pg_reg_df.index = date_x

In [21]:
pg_reg_df.to_csv('./results/2020_S/pg_reg_result.csv')